In [22]:
"""
Google News KBE Prediction Scraper - Simplified
Captures 25-50 highest-quality articles per day for predicting next-day VIX changes.
"""
import os
import hashlib
import re
import time
import random
from datetime import datetime, timedelta
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse
import pandas as pd
from pygooglenews import GoogleNews

# ============================================================================
# CONFIGURATION SECTION
# ============================================================================

# VERSION C: HYBRID (Recommended for testing)
TIER1_REGIONAL_STRESS = [
    # Specific bank names (high signal)
    '("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") AND (failure OR crisis OR collapse)',
    '("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stress OR concerns OR deposit OR loss)',
    '"New York Community Bancorp" AND (warning OR loss OR concern OR writedown)',
    
    # Regional banks generally
    '"regional bank" AND "United States" AND (failure OR crisis OR stress)',
    '"regional bank" AND (deposit OR outflow OR run)',
    
    # FDIC/emergency
    '"FDIC" AND (takeover OR receivership OR intervention)',
    '"Fed emergency" AND (facility OR liquidity)',
    '"discount window" AND surge',
]

TIER2_MAJOR_PROBLEMS = [
    # Major banks ONLY for problems
    '("JPMorgan" OR "Bank of America" OR "Wells Fargo" OR "Citigroup") AND (warning OR loss OR downgrade)',
    
    # Fed surprises
    '"Federal Reserve" AND emergency AND (meeting OR action)',
    '"FOMC" AND (surprise OR unexpected)',
    '"Jerome Powell" AND (emergency OR urgent)',
    
    # Credit market stress
    '"commercial real estate" AND (default OR distress OR crisis) AND bank',
    '"CRE loans" AND (losses OR writedown)',
    '"credit rating" AND downgrade AND bank',
]

TIER3_SECTOR_SIGNALS = [
    # Sector-wide
    '"bank earnings" AND "United States" AND (warning OR miss)',
    '"net interest margin" AND (compress OR decline)',
    '"bank reserves" AND (decline OR drain)',
    '"loan losses" AND (rising OR surge)',
    '"stress test" AND (fail OR concern)',
    '"Treasury yields" AND spike AND banks',
]


# Combine queries with priority weights
ALL_QUERIES = (
    [(q, 10) for q in TIER1_REGIONAL_STRESS] +      # 10 queries, weight 10
    [(q, 8) for q in TIER2_MAJOR_PROBLEMS] +   # 12 queries, weight 8
    [(q, 6) for q in TIER3_SECTOR_SIGNALS]         # 10 queries, weight 6
)

# API configuration
LANG = "en"
COUNTRY = "US"
MAX_ARTICLES_PER_DAY = 40  # Target 40 articles (range will be 25-50)
MAX_ARTICLES_PER_QUERY = 10  # Don't pull more than 10 from any single query

# Output directory
OUTPUT_DIR = 'data/vix_training'

# Rate limiting
MIN_DELAY = 1.0  # Minimum seconds between queries
MAX_DELAY = 2.0  # Maximum seconds between queries
ERROR_DELAY = 5  # Seconds to wait after error
DAY_DELAY_MIN = 2  # Minimum seconds between days
DAY_DELAY_MAX = 4  # Maximum seconds between days

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def clean_url(url: str) -> str:
    """Remove common tracking params to get canonical URL."""
    try:
        p = urlparse(url)
        if not p.scheme:
            return url
        q = [(k, v) for k, v in parse_qsl(p.query, keep_blank_values=True)
             if not re.match(r'^(utm_|gclid|fbclid|mc_|ref)', k, re.I)]
        return urlunparse((p.scheme, p.netloc, p.path, "", urlencode(q), ""))
    except Exception:
        return url


def make_id(title: str, canonical_link: str) -> str:
    """Generate unique ID for article based on title and link."""
    base = f"{(title or '').strip()}|{(canonical_link or '').strip()}"
    return hashlib.sha256(base.encode("utf-8")).hexdigest()


def write_daily_file(rows: list, date_str: str) -> None:
    """Write articles for a specific day to CSV."""
    if not rows:
        return
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    output_path = os.path.join(OUTPUT_DIR, f'{date_str}.csv')
    
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    print(f"  ✓ Saved {len(rows)} articles to {output_path}")


# ============================================================================
# MAIN SCRAPING FUNCTION
# ============================================================================

def run_pull_for_date(target_date: datetime, max_articles: int = 40) -> pd.DataFrame:
    """
    Pull news for a specific date with priority-based query system.
    
    Args:
        target_date: The date to pull news for (articles from this day predict VIX on next day)
        max_articles: Maximum number of articles to collect
    
    Returns:
        DataFrame of collected articles
    """
    gn = GoogleNews(lang=LANG, country=COUNTRY)
    run_iso = datetime.now().isoformat()
    
    from_date = target_date.strftime('%Y-%m-%d')
    to_date = (target_date + timedelta(days=1)).strftime('%Y-%m-%d')
    prediction_date = to_date
    
    print(f"\n{'='*70}")
    print(f"Fetching articles for {from_date}")
    print(f"(To predict VIX change on {prediction_date})")
    print(f"Target: {max_articles} articles | Max queries: {len(ALL_QUERIES)}")
    print(f"{'='*70}")
    
    collected_articles = []
    seen_ids = set()  # Track duplicates within this day only
    queries_executed = 0
    
    # Process queries in priority order
    for query_idx, (query, priority_weight) in enumerate(ALL_QUERIES, 1):
        # Stop if we've collected enough articles
        if len(collected_articles) >= max_articles:
            print(f"\n✓ Reached target of {max_articles} articles. Stopping.")
            break
        
        print(f"[{query_idx}/{len(ALL_QUERIES)}] P={priority_weight} | {query[:65]}...")
        
        try:
            # Fetch articles for this query
            res = gn.search(query, from_=from_date, to_=to_date)
            entries = res.get("entries", [])
            queries_executed += 1
            
            articles_added = 0
            
            for e in entries[:MAX_ARTICLES_PER_QUERY]:
                # Stop if we've hit the daily limit
                if len(collected_articles) >= max_articles:
                    break
                
                title = e.get("title", "")
                link = e.get("link", "")
                src = (e.get("source", {}) or {}).get("title", "")
                published = e.get("published", "")
                
                canonical = clean_url(link)
                _id = make_id(title, canonical)
                
                # Skip duplicates within this day
                if _id in seen_ids:
                    continue
                
                # Create article record
                row = {
                    "id": _id,
                    "title": title,
                    "link": canonical,
                    "published": published,
                    "source": src,
                    "query": query,
                    "priority_weight": priority_weight,
                    "article_date": from_date,
                    "prediction_date": prediction_date,
                    "fetched_at": run_iso
                }
                
                collected_articles.append(row)
                seen_ids.add(_id)
                articles_added += 1
            
            print(f"  → +{articles_added} | Total: {len(collected_articles)}/{max_articles}")
            
            # Rate limiting
            delay = random.uniform(MIN_DELAY, MAX_DELAY)
            time.sleep(delay)
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
            time.sleep(ERROR_DELAY)
            continue
    
    # Summary
    print(f"\n{'─'*70}")
    print(f"SUMMARY: {len(collected_articles)} articles from {queries_executed} queries")
    print(f"{'─'*70}")
    
    # Save results
    if collected_articles:
        write_daily_file(collected_articles, from_date)
    
    return pd.DataFrame(collected_articles)


def run_date_range(start_date: datetime, end_date: datetime, max_articles_per_day: int = 40):
    """
    Collect articles for a range of dates.
    
    Args:
        start_date: First date to collect
        end_date: Last date to collect
        max_articles_per_day: Target articles per day
    """
    current_date = start_date
    total_articles = 0
    total_queries = 0
    days_processed = 0
    start_time = time.time()
    
    total_days = (end_date - start_date).days + 1
    
    print("\n" + "="*70)
    print("KBE PREDICTION DATASET COLLECTION")
    print("="*70)
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"Total days: {total_days}")
    print(f"Target: {max_articles_per_day} articles per day")
    print(f"Query pool: {len(ALL_QUERIES)} queries (Tier 1-3)")
    print("="*70 + "\n")
    
    while current_date <= end_date:
        try:
            # Collect articles for this date
            df = run_pull_for_date(current_date, max_articles=max_articles_per_day)
            
            # Track stats
            total_articles += len(df)
            queries_used = df['query'].nunique() if len(df) > 0 else 0
            total_queries += queries_used
            days_processed += 1
            
            # Progress update every 10 days
            if days_processed % 10 == 0:
                elapsed_hours = (time.time() - start_time) / 3600
                days_remaining = (end_date - current_date).days
                queries_per_hour = total_queries / elapsed_hours if elapsed_hours > 0 else 0
                
                print(f"\n{'═'*70}")
                print(f"📊 PROGRESS UPDATE")
                print(f"{'═'*70}")
                print(f"Days: {days_processed}/{total_days} | Remaining: {days_remaining}")
                print(f"Articles: {total_articles} total ({total_articles/days_processed:.1f} avg/day)")
                print(f"Queries: {total_queries} total ({total_queries/days_processed:.1f} avg/day)")
                print(f"Rate: {queries_per_hour:.1f} queries/hour")
                print(f"Elapsed: {elapsed_hours:.2f}h | Est. remaining: {elapsed_hours/days_processed*days_remaining:.2f}h")
                print(f"{'═'*70}\n")
            
            # Short pause between days
            time.sleep(random.uniform(DAY_DELAY_MIN, DAY_DELAY_MAX))
            
        except KeyboardInterrupt:
            print("\n\n⚠️  Interrupted by user.")
            print(f"Completed: {days_processed}/{total_days} days")
            print(f"To resume, set START_DATE = datetime({current_date.year}, {current_date.month}, {current_date.day})")
            break
        except Exception as e:
            print(f"\n✗ Error processing {current_date.strftime('%Y-%m-%d')}: {e}")
            print(f"Continuing to next date...")
        
        current_date += timedelta(days=1)
    
    # Final summary
    total_time = time.time() - start_time
    print("\n" + "="*70)
    print("🎉 COLLECTION COMPLETE")
    print("="*70)
    print(f"Days processed: {days_processed}")
    print(f"Total articles: {total_articles}")
    print(f"Total queries: {total_queries}")
    print(f"Average: {total_articles/days_processed:.1f} articles/day, {total_queries/days_processed:.1f} queries/day")
    print(f"Runtime: {total_time/3600:.2f} hours")
    print(f"Query rate: {total_queries/(total_time/3600):.1f} queries/hour")
    print(f"\n📁 Output: {days_processed} CSV files in {OUTPUT_DIR}/")
    print("="*70 + "\n")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

# if __name__ == "__main__":
#     # Set your date range here
#     START_DATE = datetime(2025, 1, 1)
#     END_DATE = datetime(2025, 1, 31)
#     ARTICLES_PER_DAY = 40
    
#     print("\n🚀 Starting VIX dataset collection...")
#     print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
#     print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
#     print(f"⚡ Fast mode: 1-2 second delays\n")
    
#     # Run collection
#     run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)

In [23]:
# Jan-Feb
START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2025, 2, 28)
ARTICLES_PER_DAY = 40

print("\n🚀 Starting KBE dataset collection...")
print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
print(f"⚡ Fast mode: 1-2 second delays\n")

# Run collection
run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)


🚀 Starting KBE dataset collection...
📅 Date range: 2025-01-01 to 2025-02-28
🎯 Target: 40 articles per day
⚡ Fast mode: 1-2 second delays


KBE PREDICTION DATASET COLLECTION
Date range: 2025-01-01 to 2025-02-28
Total days: 59
Target: 40 articles per day
Query pool: 21 queries (Tier 1-3)


Fetching articles for 2025-01-01
(To predict VIX change on 2025-01-02)
Target: 40 articles | Max queries: 21
[1/21] P=10 | ("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") A...
  → +1 | Total: 1/40
[2/21] P=10 | ("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stres...
  → +0 | Total: 1/40
[3/21] P=10 | "New York Community Bancorp" AND (warning OR loss OR concern OR w...
  → +1 | Total: 2/40
[4/21] P=10 | "regional bank" AND "United States" AND (failure OR crisis OR str...
  → +0 | Total: 2/40
[5/21] P=10 | "regional bank" AND (deposit OR outflow OR run)...
  → +2 | Total: 4/40
[6/21] P=10 | "FDIC" AND (takeover OR receivership OR intervention)...
  → +0 | Total: 4/40
[7/21] 

In [24]:
# Jul - Aug
START_DATE = datetime(2025, 7, 1)
END_DATE = datetime(2025, 8, 31)
ARTICLES_PER_DAY = 40

print("\n🚀 Starting KBE dataset collection...")
print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
print(f"⚡ Fast mode: 1-2 second delays\n")

# Run collection
run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)


🚀 Starting KBE dataset collection...
📅 Date range: 2025-07-01 to 2025-08-31
🎯 Target: 40 articles per day
⚡ Fast mode: 1-2 second delays


KBE PREDICTION DATASET COLLECTION
Date range: 2025-07-01 to 2025-08-31
Total days: 62
Target: 40 articles per day
Query pool: 21 queries (Tier 1-3)


Fetching articles for 2025-07-01
(To predict VIX change on 2025-07-02)
Target: 40 articles | Max queries: 21
[1/21] P=10 | ("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") A...
  → +9 | Total: 9/40
[2/21] P=10 | ("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stres...
  → +1 | Total: 10/40
[3/21] P=10 | "New York Community Bancorp" AND (warning OR loss OR concern OR w...
  → +0 | Total: 10/40
[4/21] P=10 | "regional bank" AND "United States" AND (failure OR crisis OR str...
  → +0 | Total: 10/40
[5/21] P=10 | "regional bank" AND (deposit OR outflow OR run)...
  → +1 | Total: 11/40
[6/21] P=10 | "FDIC" AND (takeover OR receivership OR intervention)...
  → +2 | Total: 13/40
[7

In [25]:
# Sept
START_DATE = datetime(2025, 9, 1)
END_DATE = datetime(2025, 9, 30)
ARTICLES_PER_DAY = 40

print("\n🚀 Starting KBE dataset collection...")
print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
print(f"⚡ Fast mode: 1-2 second delays\n")

# Run collection
run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)


🚀 Starting KBE dataset collection...
📅 Date range: 2025-09-01 to 2025-09-30
🎯 Target: 40 articles per day
⚡ Fast mode: 1-2 second delays


KBE PREDICTION DATASET COLLECTION
Date range: 2025-09-01 to 2025-09-30
Total days: 30
Target: 40 articles per day
Query pool: 21 queries (Tier 1-3)


Fetching articles for 2025-09-01
(To predict VIX change on 2025-09-02)
Target: 40 articles | Max queries: 21
[1/21] P=10 | ("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") A...
  → +1 | Total: 1/40
[2/21] P=10 | ("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stres...
  → +0 | Total: 1/40
[3/21] P=10 | "New York Community Bancorp" AND (warning OR loss OR concern OR w...
  → +0 | Total: 1/40
[4/21] P=10 | "regional bank" AND "United States" AND (failure OR crisis OR str...
  → +0 | Total: 1/40
[5/21] P=10 | "regional bank" AND (deposit OR outflow OR run)...
  → +3 | Total: 4/40
[6/21] P=10 | "FDIC" AND (takeover OR receivership OR intervention)...
  → +0 | Total: 4/40
[7/21] 

In [26]:
# Oct - Nov
START_DATE = datetime(2025, 10, 1)
END_DATE = datetime(2025, 11, 30)
ARTICLES_PER_DAY = 40

print("\n🚀 Starting KBE dataset collection...")
print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
print(f"⚡ Fast mode: 1-2 second delays\n")

# Run collection
run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)


🚀 Starting KBE dataset collection...
📅 Date range: 2025-10-01 to 2025-11-30
🎯 Target: 40 articles per day
⚡ Fast mode: 1-2 second delays


KBE PREDICTION DATASET COLLECTION
Date range: 2025-10-01 to 2025-11-30
Total days: 61
Target: 40 articles per day
Query pool: 21 queries (Tier 1-3)


Fetching articles for 2025-10-01
(To predict VIX change on 2025-10-02)
Target: 40 articles | Max queries: 21
[1/21] P=10 | ("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") A...
  → +4 | Total: 4/40
[2/21] P=10 | ("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stres...
  → +1 | Total: 5/40
[3/21] P=10 | "New York Community Bancorp" AND (warning OR loss OR concern OR w...
  → +0 | Total: 5/40
[4/21] P=10 | "regional bank" AND "United States" AND (failure OR crisis OR str...
  → +1 | Total: 6/40
[5/21] P=10 | "regional bank" AND (deposit OR outflow OR run)...
  → +7 | Total: 13/40
[6/21] P=10 | "FDIC" AND (takeover OR receivership OR intervention)...
  → +2 | Total: 15/40
[7/21

In [27]:
# Dec
START_DATE = datetime(2025, 12, 1)
END_DATE = datetime(2025, 12, 31)
ARTICLES_PER_DAY = 40

print("\n🚀 Starting KBE dataset collection...")
print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
print(f"⚡ Fast mode: 1-2 second delays\n")

# Run collection
run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)


🚀 Starting KBE dataset collection...
📅 Date range: 2025-12-01 to 2025-12-31
🎯 Target: 40 articles per day
⚡ Fast mode: 1-2 second delays


KBE PREDICTION DATASET COLLECTION
Date range: 2025-12-01 to 2025-12-31
Total days: 31
Target: 40 articles per day
Query pool: 21 queries (Tier 1-3)


Fetching articles for 2025-12-01
(To predict VIX change on 2025-12-02)
Target: 40 articles | Max queries: 21
[1/21] P=10 | ("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") A...
  → +7 | Total: 7/40
[2/21] P=10 | ("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stres...
  → +2 | Total: 9/40
[3/21] P=10 | "New York Community Bancorp" AND (warning OR loss OR concern OR w...
  → +0 | Total: 9/40
[4/21] P=10 | "regional bank" AND "United States" AND (failure OR crisis OR str...
  → +0 | Total: 9/40
[5/21] P=10 | "regional bank" AND (deposit OR outflow OR run)...
  → +9 | Total: 18/40
[6/21] P=10 | "FDIC" AND (takeover OR receivership OR intervention)...
  → +0 | Total: 18/40
[7/21

In [28]:
# Jan 2026
START_DATE = datetime(2026, 1, 1)
END_DATE = datetime(2026, 1, 31)
ARTICLES_PER_DAY = 40

print("\n🚀 Starting KBE dataset collection...")
print(f"📅 Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 Target: {ARTICLES_PER_DAY} articles per day")
print(f"⚡ Fast mode: 1-2 second delays\n")

# Run collection
run_date_range(START_DATE, END_DATE, max_articles_per_day=ARTICLES_PER_DAY)


🚀 Starting KBE dataset collection...
📅 Date range: 2026-01-01 to 2026-01-31
🎯 Target: 40 articles per day
⚡ Fast mode: 1-2 second delays


KBE PREDICTION DATASET COLLECTION
Date range: 2026-01-01 to 2026-01-31
Total days: 31
Target: 40 articles per day
Query pool: 21 queries (Tier 1-3)


Fetching articles for 2026-01-01
(To predict VIX change on 2026-01-02)
Target: 40 articles | Max queries: 21
[1/21] P=10 | ("First Republic" OR "Silicon Valley Bank" OR "Signature Bank") A...
  → +1 | Total: 1/40
[2/21] P=10 | ("PacWest" OR "Western Alliance" OR "Zions" OR "NYCB") AND (stres...
  → +5 | Total: 6/40
[3/21] P=10 | "New York Community Bancorp" AND (warning OR loss OR concern OR w...
  → +0 | Total: 6/40
[4/21] P=10 | "regional bank" AND "United States" AND (failure OR crisis OR str...
  → +0 | Total: 6/40
[5/21] P=10 | "regional bank" AND (deposit OR outflow OR run)...
  → +8 | Total: 14/40
[6/21] P=10 | "FDIC" AND (takeover OR receivership OR intervention)...
  → +0 | Total: 14/40
[7/21